#### 1. Retrieve data from excel files

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel("import_data/FY23_Audit_必修實體及線上課程完成率.xlsx", header=None)
df_course1 = pd.read_excel("import_data/FY23_E-learning_link.xlsx", header=3, sheet_name='Experienced Hire必修線上專業課程') # course link 1
df_course2 = pd.read_excel("import_data/FY23_E-learning_link.xlsx", header=2, sheet_name='各職級必修線上專業課程') # course link 2
df_course3 = pd.read_excel("import_data/FY23_E-learning_link.xlsx", sheet_name='GCOE專業課程') # course link 3

#### 2. 處理attend_all分頁

In [ ]:
"""
attend_all tab
"""
df_empID = df.iloc[11:, 1:2].dropna()
df_empID.columns = df_empID.iloc[0]
df_empID = df_empID.drop(df_empID.index[0])

# Main
df_attend = df.iloc[:, 25:]
df_attend.drop(df_attend.index[0], inplace=True)
df_attend.drop(df_attend.index[1:8], inplace=True)                            # delete useless rows
df_attend.drop(df_attend.index[3:6], inplace=True)
df_attend.iloc[1:3] = df_attend.iloc[1:3].ffill(axis=1)                         # 處理置中- fill course name & course number(to the right)
df_attend = df_attend.replace(r'\n',' ', regex=True)                          # replace new line 
df_attend.iloc[0,:] = df_attend.replace(np.nan, '否').bfill().iloc[0,:]
df_attend.columns = df_attend.iloc[0,:].apply(str) +'!'+ df_attend.iloc[1,:].apply(str) +'!'+ df_attend.iloc[2,:].apply(str)+'!'+ df_attend.iloc[3,:].apply(str)
df_attend.drop(df_attend.index[0:4], inplace=True)                            # remove first 3 rows
df_attend = df_attend.rename(columns = {df_attend.columns[0]:'員工編號'})              
df_attend['員工編號']  = df_empID['員工編號']                                # merge employee id
df_attend = df_attend[df_attend['員工編號'].notna()].fillna(0.0)
df_attend = df_attend.set_index('員工編號').stack().reset_index()               # transform table
df_attend[['前一年度Plan是否有','course_number', '課程名稱', 'hours']] = df_attend.level_1.str.split('!', expand=True)
df_attend = df_attend.drop(columns='level_1')
df_attend = df_attend[df_attend['course_number'].str.contains("---|TBD") == False]     # future warning!!!
df_attend_toPiv = df_attend.copy()

df_plan = df_attend[['course_number', '前一年度Plan是否有']].drop_duplicates(subset="course_number")

df_attend_toPiv = df_attend_toPiv.pivot(index=['員工編號','course_number', '課程名稱'],
                                        columns='hours',
                                        values=0).reset_index(drop=False)

df_attend_toPiv = df_attend_toPiv.rename(columns = {'實際時數':'實上', '必修時數(應上)':'應上'})
res_df_attend = df_attend_toPiv.merge(df_plan, how='left', on='course_number')


need_vals = ['取消(年休假/事假/其他)','取消(因公)','取消(預計離職)','待完成','待補課']   # 列入完成率計算
noNeed_vals= ['取消(婚假/產假)','取消(病假)','取消(喪假)','取消(公假/溫書假)']          # 不列入完成率計算

res_df_attend.loc[res_df_attend['實上'].isin(noNeed_vals), '應上'] = 0
res_df_attend.loc[res_df_attend['實上'].isin(need_vals + noNeed_vals), '實上'] = 0

res_df_attend = (res_df_attend.drop(['應上'], axis=1).join(res_df_attend['應上'].apply(pd.to_numeric, errors='coerce')))
res_df_attend['實上'] = np.where(res_df_attend['實上'].map(np.isreal) == False, res_df_attend['應上'], res_df_attend['實上'])  #處理梯次+奇怪的值...轉成[應上]的值
res_df_attend = (res_df_attend.drop(['實上'], axis=1).join(res_df_attend['實上'].apply(pd.to_numeric, errors='coerce')))

res_df_attend = res_df_attend.rename(columns={'前一年度Plan是否有':'年度'})
res_df_attend.loc[res_df_attend['年度'] == '是', '年度'] = '前年度'
res_df_attend.loc[res_df_attend['年度'] == '否', '年度'] = '今年度'
res_df_attend['備註'] = np.where(res_df_attend['應上'] - res_df_attend['實上'] != 0, '待完成', '')

#### 3. 處理Employee分頁

In [ ]:
"""
EMPLOYEE tab
"""
df_emp = df.iloc[14:, :16] # row, column

df_emp.columns = df_emp.iloc[0,:]
df_emp = df_emp.drop(df_emp.index[0])
df_emp = df_emp.drop(df_emp.columns[[7, 8, 13, 14]], axis=1)

# FY23未升遷
df_emp['FY23未升遷'] = df_emp['FY23未升遷'].fillna('有升遷')
df_emp = df_emp.rename(columns = {df_emp.columns[-5]:'是否升遷'})

# GCoE學員
df_emp['GCoE學員'] = df_emp['GCoE學員'].fillna('否')
df_emp.loc[df_emp['GCoE學員'] == 'Yes', 'GCoE學員'] = '是'

# 前一年度取得資格者
df_emp['前一年度取得資格者'] = df_emp['前一年度取得資格者'].fillna('否')
df_emp.loc[df_emp['前一年度取得資格者'] == 'Yes', '前一年度取得資格者'] = '是'

# A20B對應Audit職級要排的課
df_emp['A20B對應Audit職級要排的課'] = df_emp['A20B對應Audit職級要排的課'].fillna('')

# 年度中到職(12/1以後)
df_emp['年度中到職(12/1以後)'] = df_emp['年度中到職(12/1以後)'].fillna('否')
df_emp.loc[df_emp['年度中到職(12/1以後)'] != '否', '年度中到職(12/1以後)'] = '是。12/1-4/30到職者，至少須完成必修時數10小時。'